In [93]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import time

In [53]:
df = pd.read_csv('../clean/BoxOffice_clean.csv')

In [54]:
df.head()

,Unnamed: 0,Year,Rank,Movie,Worldwide,Domestic,%Domestic,Foreign,%Foreign
0,0,2019,1,Avengers: Endgame,2797800564,858373000.0,30.70,1.939428e+09,69.3
1,1,2019,2,The Lion King,1655146969,543486894.0,32.80,1.111660e+09,67.2
2,2,2019,3,Spider-Man: Far From Home,1131845802,390532085.0,34.50,7.413137e+08,65.5
3,3,2019,4,Captain Marvel,1128274794,426829839.0,37.80,7.014450e+08,62.2
4,4,2019,5,Toy Story 4,1073163682,433952080.0,40.40,6.392116e+08,59.6


In [55]:
df.shape

(1200, 9)

In [100]:
def format_string(s):
    return s.replace(" ", "+")

In [148]:
apikey = '2ed31365'

'2ed31365'

Have to use 2 APIs here since each API has 1k Daily limit

In [149]:
# jsons = []
jsons2 = []
# for i, m in enumerate(df['Movie'][:700]):
for i, m in enumerate(df['Movie'][700:]):
    if i % 50 == 0:
        time.sleep(5)
    title = format_string(m)
    link = 'http://www.omdbapi.com/?t={}&apikey={}'.format(title,apikey)
    response = requests.get(link)
    cur_str = response.content.decode('utf-8')
    json_d = json.loads(cur_str)
#     jsons.append(json_d)
    jsons2.append(json_d)

In [147]:
len(jsons)

700

In [151]:
len(jsons2)

500

In [154]:
all_jsons = jsons+jsons2

In [155]:
len(all_jsons)

1200

In [161]:
df_output = pd.DataFrame(all_jsons)
df_output.head()

,Actors,Awards,BoxOffice,Country,DVD,Director,Error,Genre,Language,Metascore,...,Runtime,Title,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,totalSeasons
0,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",N/A,N/A,USA,30 Jul 2019,"Anthony Russo, Joe Russo",NaN,"Action, Adventure, Drama, Sci-Fi","English, Japanese, Xhosa",78,...,181 min,Avengers: Endgame,movie,N/A,"Christopher Markus (screenplay by), Stephen Mc...",2019,tt4154796,8.5,"589,503",NaN
1,"Rowan Atkinson, Matthew Broderick, Niketa Cala...",Won 2 Oscars. Another 33 wins & 29 nominations.,"$94,240,635",USA,07 Oct 2003,"Roger Allers, Rob Minkoff",NaN,"Animation, Adventure, Drama, Family, Musical","English, Swahili, Xhosa, Zulu",88,...,88 min,The Lion King,movie,N/A,"Irene Mecchi (screenplay by), Jonathan Roberts...",1994,tt0110357,8.5,"869,323",NaN
2,"Tom Holland, Samuel L. Jackson, Jake Gyllenhaa...",N/A,N/A,USA,17 Sep 2019,Jon Watts,NaN,"Action, Adventure, Sci-Fi","Italian, Czech, English",69,...,129 min,Spider-Man: Far from Home,movie,N/A,"Chris McKenna, Erik Sommers, Stan Lee (based o...",2019,tt6320628,7.6,"221,516",NaN
3,"Brie Larson, Samuel L. Jackson, Ben Mendelsohn...",N/A,N/A,"USA, Australia",11 Jun 2019,"Anna Boden, Ryan Fleck",NaN,"Action, Adventure, Sci-Fi",English,64,...,123 min,Captain Marvel,movie,N/A,"Anna Boden (screenplay by), Ryan Fleck (screen...",2019,tt4154664,7.0,"368,679",NaN
4,"Tom Hanks, Tim Allen, Annie Potts, Tony Hale",N/A,N/A,USA,01 Oct 2019,Josh Cooley,NaN,"Animation, Adventure, Comedy, Family, Fantasy",English,84,...,100 min,Toy Story 4,movie,N/A,"John Lasseter (original story by), Andrew Stan...",2019,tt1979376,8.0,"121,373",NaN


## Getting List of Marvel/DC Movies

In [184]:
with open('../clean/mcu_movies.txt','r') as f:
    myfile_mv = f.read()
with open('../clean/dc_movies.txt','r') as f:
    myfile_dc = f.read()    

In [185]:
mv_names = myfile_mv.split('\n')
dc_names = myfile_dc.split('\n')

In [186]:
mv_names = [m.strip() for m in mv_names]
dc_names = [m.strip() for m in dc_names]
mv_names

['Iron Man',
 'Iron Man 2',
 'Iron Man 3',
 'The Incredible Hulk',
 'Thor',
 'Thor: The Dark World',
 'Thor: Ragnarok',
 'Captain America: The First Avenger',
 'Captain America: The Winter Soldier',
 'Captain America: Civil War',
 'The Avengers',
 'Avengers: Age of Ultron',
 'Avengers: Infinity War',
 'Avengers: Endgame',
 'Guardians of the Galaxy',
 'Guardians of the Galaxy Vol. 2',
 'Ant-Man',
 'Ant-Man and the Wasp',
 'Doctor Strange',
 'Spider-Man: Homecoming',
 'Spider-Man: Far from Home',
 'Black Panther',
 'Captain Marvel']

In [187]:
movies_mv = set(mv_names)
movies_dc = set(dc_names)

In [188]:
len(movies_mv),len(movies_dc)

(23, 13)

In [189]:
movies_dc

{'Aquaman',
 'Batman v Superman: Dawn of Justice',
 'Green Lantern',
 'Joker',
 'Jonah Hex',
 'Justice League',
 'Man of Steel',
 'Shazam!',
 'Suicide Squad',
 'The Dark Knight',
 'The Dark Knight Rises',
 'Watchmen',
 'Wonder Woman'}

## Appending Marvel/DC Tags

In [205]:
genre_marvel = []
genre_dc = []
matches_mv = set()
matches_dc = set()
missing_indices = []
for k,v in df_output.iterrows():
    title = v['Title']
#     if type(title) == float:
#         missing_indices.append(k)
    
#     if type(title) == str and title[:5] == "Jonah":
#         print(k,v)
    if title in movies_mv:
        genre_marvel.append(1)
        matches_mv.add(title)
    else:
        genre_marvel.append(0)
        
    if title in movies_dc:
        genre_dc.append(1)
        matches_dc.add(title)
    else:
        genre_dc.append(0)        

In [211]:
df.loc[missing_indices]

,Unnamed: 0,Year,Rank,Movie,Worldwide,Domestic,%Domestic,Foreign,%Foreign
15,15,2019,16,"My People, My Country",420105485,2356683.0,0.60,417748802.0,99.4
188,188,2018,89,Spirited Away 2018 Re-release,70186029,1175067.0,1.70,69010962.0,98.3
274,274,2017,75,Atomic Blonde 2017 Re-release,100014025,51687870.0,51.70,48326155.0,48.3
480,480,2015,81,Kill Zone 2,93638358,11998.0,<0.1,93626360.0,100.0
654,654,2013,55,The Wind Rises 2013 Re-release,136360863,5209580.0,3.80,131151283.0,96.2
661,661,2013,62,Jurassic Park 2013 Re-release,116485935,45385935.0,39,71100000.0,61.0
720,720,2012,21,Titanic 2012 3D Release,343550770,57884114.0,16.80,285666656.0,83.2
766,766,2012,67,Star Wars: Episode I - The Phantom Menace 2012...,102739593,43456382.0,42.30,59283211.0,57.7
785,785,2012,86,Finding Nemo 2012 3D Release,72128283,41128283.0,57,31000000.0,43.0
786,786,2012,87,The Cabin in the Woods 2012 Re-release,69935600,42073277.0,60.20,27862323.0,39.8


In [196]:
# np.sum(genre_marvel)
# np.sum(genre_dc)
movies_dc-matches_dc

{'Jonah Hex'}

In [212]:
len(genre_marvel)

1200

In [213]:
len(genre_dc)

1200

In [214]:
df_output['Marvel Film'] = genre_marvel
df_output['DC Film'] = genre_dc

In [215]:
df_output.head()

,Actors,Awards,BoxOffice,Country,DVD,Director,Error,Genre,Language,Metascore,...,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,totalSeasons,Marvel Film,DC Film
0,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",N/A,N/A,USA,30 Jul 2019,"Anthony Russo, Joe Russo",NaN,"Action, Adventure, Drama, Sci-Fi","English, Japanese, Xhosa",78,...,movie,N/A,"Christopher Markus (screenplay by), Stephen Mc...",2019,tt4154796,8.5,"589,503",NaN,1,0
1,"Rowan Atkinson, Matthew Broderick, Niketa Cala...",Won 2 Oscars. Another 33 wins & 29 nominations.,"$94,240,635",USA,07 Oct 2003,"Roger Allers, Rob Minkoff",NaN,"Animation, Adventure, Drama, Family, Musical","English, Swahili, Xhosa, Zulu",88,...,movie,N/A,"Irene Mecchi (screenplay by), Jonathan Roberts...",1994,tt0110357,8.5,"869,323",NaN,0,0
2,"Tom Holland, Samuel L. Jackson, Jake Gyllenhaa...",N/A,N/A,USA,17 Sep 2019,Jon Watts,NaN,"Action, Adventure, Sci-Fi","Italian, Czech, English",69,...,movie,N/A,"Chris McKenna, Erik Sommers, Stan Lee (based o...",2019,tt6320628,7.6,"221,516",NaN,1,0
3,"Brie Larson, Samuel L. Jackson, Ben Mendelsohn...",N/A,N/A,"USA, Australia",11 Jun 2019,"Anna Boden, Ryan Fleck",NaN,"Action, Adventure, Sci-Fi",English,64,...,movie,N/A,"Anna Boden (screenplay by), Ryan Fleck (screen...",2019,tt4154664,7.0,"368,679",NaN,1,0
4,"Tom Hanks, Tim Allen, Annie Potts, Tony Hale",N/A,N/A,USA,01 Oct 2019,Josh Cooley,NaN,"Animation, Adventure, Comedy, Family, Fantasy",English,84,...,movie,N/A,"John Lasseter (original story by), Andrew Stan...",2019,tt1979376,8.0,"121,373",NaN,0,0


In [219]:
# ! ls ../raw

BoxOffice.csv            BoxOffice.xlsx           all_character_pages.json


In [220]:
df_output.to_csv('../raw/genre_and_reviews_omdb.csv')